In [ ]:
import os
import requests
import zipfile

# dataset url
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"
filename = "data.zip"

print("1. Downloading data...")
if not os.path.exists(filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print("   Download done.")
else:
    print("   File is already exist, passing download process.")

print("2. Extracting data...")
if not os.path.exists('data'):
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("   Extraction complete. 'data' folder is ready.")
else:
    print(" 'data' folder already exist.")

1. Downloading data...
   Download done.
2. Extracting data...
   Extraction complete. 'data' folder is ready.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# set seed
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# (GPU NVIDIA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
class HairNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Conv Layer: 3 input channels (RGB), 32 output filters, kernel 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        
        # Max Pool: size 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Linear Layers
        # Flatten calculation:
        # Input: 200x200 -> Conv(3x3) -> 198x198
        # Pool(2x2) -> 99x99
        # Total pixels: 32 channel * 99 * 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Convolution -> ReLU -> MaxPool
        x = self.pool(F.relu(self.conv1(x)))
        
        # Flatten
        x = torch.flatten(x, 1)
        
        # Linear -> ReLU
        x = F.relu(self.fc1(x))
        
        # Output (Logits)
        x = self.fc2(x)
        return x

model = HairNet().to(device)

# --- q1 & q2 answers ---
# q1: loss function for binary classification with 1 neuron output (without sigmoid in forward)
criterion = nn.BCEWithLogitsLoss() 
print(f"Q1: Use nn.BCEWithLogitsLoss()")

# Q2: calculate parameter
total_params = sum(p.numel() for p in model.parameters())
print(f"Q2 Total parameters: {total_params}")

Q1: Use nn.BCEWithLogitsLoss()
Q2 Total parameters: 20073473


In [ ]:
# transformation
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = train_transforms

# dataset
train_dataset = datasets.ImageFolder('./data/train', transform=train_transforms)
validation_dataset = datasets.ImageFolder('./data/test', transform=test_transforms)

# dataLoader (num_workers=0 for windows stabilization)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=0)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False, num_workers=0)

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# --- TRAINING LOOP ---
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

print("start training part 1...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) 

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # manual sigmoid because using BCEWithLogitsLoss
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# --- q3 & q4 answers ---
median_acc = np.median(history['acc'])
std_loss = np.std(history['loss'])

print(f"\nQ3 Median Training Accuracy: {median_acc:.2f}")
print(f"Q4 Std Training Loss: {std_loss:.4f}")

start training part 1...
Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015

Q3 Median Training Accuracy: 0.82
Q4 Std Training Loss: 0.1590


In [ ]:
# 1. transformation update (augmentation)
aug_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. dataset & new loader 
train_dataset_aug = datasets.ImageFolder('./data/train', transform=aug_transforms)
# num_workers=0 for windows safety
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=0)

# history reset for second session
history_aug = {'val_loss': [], 'val_acc': []}

print("\ncontinue training with augmentation (Part 2)...")

# 3. continue training (10 Epoch more)
for epoch in range(10):
    model.train()
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # validation (use old/without augmentation validation loader)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/10, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# --- q5 & q6 answers ---
mean_val_loss = np.mean(history_aug['val_loss'])
avg_val_acc_last5 = np.mean(history_aug['val_acc'][5:])

print(f"\nQ5 Mean Validation Loss: {mean_val_loss:.4f}")
print(f"Q6 Avg Val Acc (last 5 epochs): {avg_val_acc_last5:.2f}")


continue training with augmentation (Part 2)...
Aug Epoch 1/10, Val Loss: 0.6577, Val Acc: 0.6816
Aug Epoch 2/10, Val Loss: 0.7313, Val Acc: 0.6766
Aug Epoch 3/10, Val Loss: 0.5772, Val Acc: 0.6965
Aug Epoch 4/10, Val Loss: 0.5688, Val Acc: 0.7015
Aug Epoch 5/10, Val Loss: 0.6278, Val Acc: 0.6567
Aug Epoch 6/10, Val Loss: 0.5948, Val Acc: 0.7015
Aug Epoch 7/10, Val Loss: 0.7839, Val Acc: 0.5970
Aug Epoch 8/10, Val Loss: 0.8211, Val Acc: 0.6318
Aug Epoch 9/10, Val Loss: 0.5448, Val Acc: 0.7114
Aug Epoch 10/10, Val Loss: 0.5355, Val Acc: 0.7463

Q5 Mean Validation Loss: 0.6443
Q6 Avg Val Acc (last 5 epochs): 0.68
